In [19]:
import urllib.request
from bs4 import BeautifulSoup

def GetAllDoxyDonkeyPosts(url, links):
    request=urllib.request.Request(url)
    response=urllib.request.urlopen(request)

    soup=BeautifulSoup(response)
    for a in soup.findAll('a'):
        try:
            url=a['href']
            title=a['title']
            if title =="Older Posts":
                print(title, url)
                links.append(url)
                GetAllDoxyDonkeyPosts(url, links)
        except:
            title=""
    return

In [20]:
blogUrl="http://doxydonkey.blogspot.in"
links=[]
GetAllDoxyDonkeyPosts(blogUrl, links)

c:\python35\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\python35\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=7&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=14&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=21&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=28&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=35&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-02-21T19:13:00-08:00&max-results=7&start=42&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-02-12T18:34:00-08:00&max-results=7&start=49&by-date=false
Older Posts http://doxydonkey.bl

In [30]:
def GetDoxyDonkeyText(testUrl):
    request=urllib.request.Request(testUrl)
    response=urllib.request.urlopen(request)
    soup=BeautifulSoup(response, "lxml")
    myDivs=soup.findAll("div", {'class':'post-body'})
    posts=[]
    for div in myDivs:
        posts+=map(lambda  p:p.text.encode('ascii', errors='replace').replace(b"?", b" "), div.findAll("li"))
    return posts

In [31]:
doxyDonkeyPosts=[]

In [33]:
for link in links:
    doxyDonkeyPosts+=GetDoxyDonkeyText(link)

In [72]:
doxyDonkeyPostsString=[]
for i, sent in enumerate(doxyDonkeyPosts):
    doxyDonkeyPostsString.append(str(doxyDonkeyPosts[i], 'utf8'))

In [73]:
doxyDonkeyPostsString[0]

'Why did ESPN let go of 100 of its writers, reporters and on-air staff today  The programmer, which has around 8,000 employees, says it s making the moves as it adapts its mix of TV and digital programming to the Twitter/Facebook/Snapchat age. Its content costs are rising as it pays ever-increasing fees for rights to show college and pro sports. But its subscriber base is shrinking as pay TV customers cut the cord or never sign up for it in the first place. The short version of his answer: ESPN thinks it will continue to grow its subscriber revenue by charging its remaining subscribers (via pay TV distributors) more for the service, and that it can keep growing ad rates, too. But ESPN can t simply grow its way out of this problem. It will have to cut costs, too.'

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [83]:
vectorizer=TfidfVectorizer(max_df=0.5, min_df=0.2, stop_words='english')

In [84]:
X=vectorizer.fit_transform(doxyDonkeyPostsString)

In [85]:
X

<3528x26 sparse matrix of type '<class 'numpy.float64'>'
	with 25622 stored elements in Compressed Sparse Row format>

In [86]:
print(X[0])

  (0, 18)	0.692788764518
  (0, 20)	0.7211405742


In [88]:
from sklearn.cluster import KMeans

In [89]:
kmean=KMeans(n_clusters=3, init='k-means++', n_init=1, max_iter=300, verbose=True)

In [90]:
kmean.fit(X)

Initialization complete
Iteration  0, inertia 4030.774
Iteration  1, inertia 2437.891
Iteration  2, inertia 2397.577
Iteration  3, inertia 2385.961
Iteration  4, inertia 2379.547
Iteration  5, inertia 2376.665
Iteration  6, inertia 2375.543
Iteration  7, inertia 2374.633
Iteration  8, inertia 2373.887
Iteration  9, inertia 2373.564
Iteration 10, inertia 2373.233
Iteration 11, inertia 2373.031
Iteration 12, inertia 2372.884
Iteration 13, inertia 2372.737
Iteration 14, inertia 2372.524
Iteration 15, inertia 2372.358
Iteration 16, inertia 2372.255
Iteration 17, inertia 2372.172
Iteration 18, inertia 2372.119
Iteration 19, inertia 2372.104
Converged at iteration 19: center shift 2.524785e-06 within tolerance 2.916244e-06


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [91]:
import numpy as np
np.unique(kmean.labels_, return_counts=True)

(array([0, 1, 2]), array([1306, 1159, 1063], dtype=int64))

In [94]:
len(kmean.labels_)

3528

In [97]:
text={}
for i, cluster in enumerate(kmean.labels_):
    oneDocument=doxyDonkeyPostsString[i]
    if cluster not in text.keys():
        text[cluster]=oneDocument
    else:
        text[cluster]+=oneDocument

In [98]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk 

In [100]:
_stopwords = set(stopwords.words('english') + list(punctuation)+["million","billion","year","millions","billions","y/y","'s","''","``"])

In [101]:
keywords={}
counts={}
for cluster in range(3):
    word_sent=word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq=FreqDist(word_sent)
    keywords[cluster]=nlargest(100, freq, key=freq.get)
    counts[cluster]=freq

In [103]:
uniqe_keys={}
for cluster in range(3):
    other_clusters=list(set(range(3))-set([cluster]))
    key_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique=set(keywords[cluster])-key_other_clusters
    uniqe_keys[cluster]=nlargest(10, unique,key=counts[cluster].get)

In [105]:
print(uniqe_keys)

{0: ['revenue', 'quarter', 'share', 'profit', 'rose', 'analysts', 'per', 'reported', '1', 'earnings'], 1: ['mr.', 'industry', 'work', 'car', 'part', 'next', 'venture', 'cars', 'well', 'take'], 2: ['ads', 'apps', 'social', 'ad', 'search', 'user', 'snapchat', 'platform', 'using', 'news']}


In [118]:
newArticle="Amazon.com Inc launched Echo Show, a touchscreen device that will allow users to video call and watch clips from CNN, the latest in the company's series of popular Echo voice-controlled speakers. The device, which will go on sale in June for about $230, will feature Alexa, Amazon's voice-controlled aide, that can be used to play music, order an Uber or turn on the house lights. Echo Show will allow video conferencing between users having an Echo device or the Alexa app. It is the first to support the feature, which is absent in similar devices offered by rivals such as Alphabet Inc's unit Google."

In [107]:
from sklearn.neighbors import KNeighborsClassifier

In [108]:
classifier=KNeighborsClassifier()

In [109]:
classifier.fit(X, kmean.labels_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [119]:
test=vectorizer.transform([newArticle.encode('ascii',errors='ignore')])

In [120]:
classifier.predict(test)

array([2])